Cincinnati Bengals - Code Challenge

In [ ]:
# imports
import boto3
import pandas as pd
import psycopg2 
from sqlalchemy import create_engine

In [ ]:
# constants
HOST = 'ls-2619b6b15c9bdc80a23f6afb7eee54cf0247da21.ca3yee6xneaj.us-east-1.rds.amazonaws.com'
PORT="5432"
REGION="us-east-1"
DATABASE = 'postgres'
USER = 'danny_sostre'
PW = 'sannydostre'
TABLE = 'danny_sostre'

AWS_BUCKET = 'mindex-data-analytics-code-challenge'

BENGALS = 'bengals.csv'
BOYD = 'boyd_receiving.csv'
CHASE = 'chase_receiving.csv'
HIGGINS = 'higgins_receiving.csv'

In [ ]:
# function to download aws file
def download_aws_file(bucket, filename, new_filename):
    s3 = boto3.client('s3')
    s3.download_file(bucket, filename, new_filename)

In [ ]:
# function to download all files neccessary for project
def download_aws_files():
    download_aws_file(AWS_BUCKET, BENGALS, BENGALS)
    download_aws_file(AWS_BUCKET, BOYD, BOYD)
    download_aws_file(AWS_BUCKET, CHASE, CHASE)
    download_aws_file(AWS_BUCKET, HIGGINS, HIGGINS)

In [ ]:
# function to get dataframe from file
def get_df_from_file(filename):
    df = pd.read_csv (filename)
    return df

In [ ]:
# function to merge all 4 stat files -- outer join
def get_merged_df():

    csv_bengals = get_df_from_file(BENGALS)

    csv_boyd = get_df_from_file(BOYD)
    csv_boyd.rename(columns={'Yards': 'YardsBoyd', 'TD': 'TD_Boyd'}, inplace = True)

    csv_chase = get_df_from_file(CHASE)
    csv_chase.rename(columns={'Yards': 'YardsChase', 'TD': 'TD_Chase'}, inplace = True)

    csv_higgins = get_df_from_file(HIGGINS)
    csv_higgins.rename(columns={'Yards': 'YardsHiggins', 'TD': 'TD_Higgins'}, inplace = True)

    merged_df = pd.merge(csv_bengals, pd.merge(csv_boyd, pd.merge(csv_chase, csv_higgins, on="Week", how='outer'), on="Week", how='outer'), on="Week", how='outer')
    merged_df['Result'] = merged_df['Result'].replace([1.0, 0.0],['Win', 'Loss'])
    return merged_df

In [ ]:
# get records from database
def get_records (select_sql):
    connection = None
    try:
        connection = psycopg2.connect(
            host=HOST,
            database=DATABASE,
            user=USER,
            password=PW)
        cursor = connection.cursor()

        cursor.execute(select_sql)
        results = cursor.fetchall()
        return results

    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [ ]:
# insert records from dataframe
def insert_dataframe (df):    
    connection = None
    conn_string = 'postgresql://' + USER + ':' + PW + '@' + HOST + '/' + DATABASE
    db = create_engine(conn_string)
    try:
        connection = db.connect()
        df.to_sql(TABLE, con=connection, if_exists='replace', index=True)
        connection = psycopg2.connect(conn_string)
        connection.autocommit = True
        cursor = connection.cursor()  

    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")  

In [ ]:
# get files
download_aws_files()

#merge dataframes
merged_df = get_merged_df()
insert_dataframe(merged_df)

# get records from db and print
records = get_records('SELECT * FROM ' + TABLE)
for r in records:
    print(r)